<a href="https://colab.research.google.com/github/coder-royswarnajit/RNN__Q-A_Bot/blob/main/RNN__Q%26A_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df=pd.read_csv("/content/100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [ ]:
#Tokenizing
def tokenize(text):
  text=text.lower().replace('?','').replace("'","").split()
  return text

tokenize("Hi I am Roy")

['hi', 'i', 'am', 'roy']

In [ ]:
#Vocab
vocab={'<UNK>':0}
def build_vocab(row):
  Question=tokenize(row['question'])
  Answer=tokenize(row['answer'])
  merged=Question + Answer

  for token in merged:
    if token not in vocab:
      vocab[token]=len(vocab)

df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [ ]:
len(vocab)

324

In [ ]:
#Text into Indices
def text_indces(text,vocab):
  indexed_text=[]
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [ ]:
#Building Dataset Class
import torch
from torch.utils.data import Dataset,DataLoader

class QADataset(Dataset):
  def __init__(self,df,vocab):
    self.df=df
    self.vocab=vocab

  def __len__(self):
    return len(self.df)

  def __getitem__(self,index):
    Question=text_indces(self.df.iloc[index]['question'],self.vocab)
    Answer=text_indces(self.df.iloc[index]['answer'],self.vocab)

    return torch.tensor(Question),torch.tensor(Answer)

dataset=QADataset(df,vocab)
dataloader=DataLoader(dataset,batch_size=1,shuffle=True)

In [ ]:
#RNN Network
import torch.nn as nn

class Rnn(nn.Module):
  def __init__(self,vocab_size,embedding_size,hidden_size):
    super().__init__()
    self.embeddings=nn.Embedding(vocab_size,embedding_size)
    self.rnn=nn.RNN(embedding_size,hidden_size,batch_first=True)
    self.fc=nn.Linear(hidden_size,vocab_size)

  def forward(self,question):
    _,h=self.rnn(self.embeddings(question))
    output=self.fc(h.squeeze(0))
    return output

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 12.2 MB/s eta 0:00:00


In [ ]:
#Model
import optuna

def objective(trial):
  embedding_size=trial.suggest_int('embedding_size',32,128)
  hidden_size=trial.suggest_int('hidden_size',64,256)
  lr=trial.suggest_float('lr',1e-4,1e-2,log=True)
  epochs=trial.suggest_int('epochs',10,30)


  model=Rnn(len(vocab),embedding_size,hidden_size)
  criterion=nn.CrossEntropyLoss()
  optimizer=torch.optim.Adam(model.parameters(),lr=lr)

  for epoch in range(epochs):
    total_loss = 0
    for question, answer in dataloader:
      optimizer.zero_grad()
      output=model(question)
      loss=criterion(output,answer[0])
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

    total_loss += total_loss / len(dataloader)
    print(f"Epoch: {epoch+1}, Loss: {total_loss}")

  return total_loss / epochs


In [ ]:
study=optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=30)

[I 2026-02-03 13:29:11,359] A new study created in memory with name: no-name-86771b49-02bb-40bd-9715-0ec9c0bbc993


Epoch: 1, Loss: 560.9975555234486
Epoch: 2, Loss: 553.4345868640476
Epoch: 3, Loss: 531.7318411913183
Epoch: 4, Loss: 495.8686039799824
Epoch: 5, Loss: 393.4095231365412
Epoch: 6, Loss: 302.64120321385565
Epoch: 7, Loss: 221.26593937931142
Epoch: 8, Loss: 170.17457233969148
Epoch: 9, Loss: 139.70164577346173
Epoch: 10, Loss: 73.39132939206615
Epoch: 11, Loss: 80.82038084668856
Epoch: 12, Loss: 68.78541258015515
Epoch: 13, Loss: 67.97350757689638
Epoch: 14, Loss: 42.1435727963469


[I 2026-02-03 13:29:15,244] Trial 0 finished with value: 2.453757988625813 and parameters: {'embedding_size': 40, 'hidden_size': 211, 'lr': 0.00564421166176214, 'epochs': 15}. Best is trial 0 with value: 2.453757988625813.


Epoch: 15, Loss: 36.8063698293872
Epoch: 1, Loss: 534.4420682059394
Epoch: 2, Loss: 287.9490117258496
Epoch: 3, Loss: 109.12191424783734
Epoch: 4, Loss: 49.76567754333632
Epoch: 5, Loss: 22.05642591905263
Epoch: 6, Loss: 7.8872740487257635
Epoch: 7, Loss: 3.4075182574076783
Epoch: 8, Loss: 2.0912614522501825
Epoch: 9, Loss: 1.6241097546337793
Epoch: 10, Loss: 1.3535065426594681
Epoch: 11, Loss: 1.1560121940966281
Epoch: 12, Loss: 0.9974752019159496
Epoch: 13, Loss: 0.8722311796672229
Epoch: 14, Loss: 0.772442436714967


[I 2026-02-03 13:29:20,293] Trial 1 finished with value: 0.04558016269519512 and parameters: {'embedding_size': 66, 'hidden_size': 243, 'lr': 0.0017591139953620691, 'epochs': 15}. Best is trial 1 with value: 0.04558016269519512.


...

Epoch: 30, Loss: 24.427435536077084
Epoch: 1, Loss: 531.9375524944729
Epoch: 2, Loss: 292.2891772614585
Epoch: 3, Loss: 110.19574319753382
Epoch: 4, Loss: 38.35980804603961
Epoch: 5, Loss: 18.89901723836859
Epoch: 6, Loss: 13.706950737059945
Epoch: 7, Loss: 7.3368108530218406
Epoch: 8, Loss: 4.641327659785747
Epoch: 9, Loss: 2.8893266330783565
Epoch: 10, Loss: 2.225207908782694
Epoch: 11, Loss: 1.8392867170377738
Epoch: 12, Loss: 1.5438515547352532
Epoch: 13, Loss: 1.3199313507518835
Epoch: 14, Loss: 1.1437836398681005
Epoch: 15, Loss: 1.0026549444068222
Epoch: 16, Loss: 0.8837831967717243
Epoch: 17, Loss: 0.7864467928962161
Epoch: 18, Loss: 0.7027960730406145
Epoch: 19, Loss: 0.6306003351359525
Epoch: 20, Loss: 0.571408918300747
Epoch: 21, Loss: 0.5182520607486367
Epoch: 22, Loss: 0.471420287164963
Epoch: 23, Loss: 0.430428665674602
Epoch: 24, Loss: 0.3948397679176802
Epoch: 25, Loss: 0.36295107863326037
Epoch: 26, Loss: 0.333763918479801
Epoch: 27, Loss: 0.30840073668708406


[I 2026-02-03 13:32:27,994] Trial 28 finished with value: 0.010177907007164323 and parameters: {'embedding_size': 99, 'hidden_size': 218, 'lr': 0.0013162622349506762, 'epochs': 28}. Best is trial 12 with value: 0.0020991424264610505.


Epoch: 28, Loss: 0.28498139620060103
Epoch: 1, Loss: 528.4968123806848
Epoch: 2, Loss: 377.3518634107378
Epoch: 3, Loss: 244.6813117702802
Epoch: 4, Loss: 143.10689601037237
Epoch: 5, Loss: 78.79564091281759
Epoch: 6, Loss: 45.152007135252155
Epoch: 7, Loss: 27.768227423975866
Epoch: 8, Loss: 18.82870598650641
Epoch: 9, Loss: 12.788366282896863
Epoch: 10, Loss: 8.944040138026079
Epoch: 11, Loss: 6.851781842692031


[I 2026-02-03 13:32:32,062] Trial 29 finished with value: 0.4600606486563467 and parameters: {'embedding_size': 86, 'hidden_size': 235, 'lr': 0.0006362608060594889, 'epochs': 12}. Best is trial 12 with value: 0.0020991424264610505.


Epoch: 12, Loss: 5.52072778387616


In [ ]:
print("Best params:", study.best_params)

Best params: {'embedding_size': 90, 'hidden_size': 221, 'lr': 0.004067677787459624, 'epochs': 30}


In [ ]:
#prediction
def predict(model,question,threshold=0.5):
  question=text_indces(question,vocab)
  question_tensor=torch.tensor(question).unsqueeze(0)
  output=model(question_tensor)
  probs=torch.nn.functional.softmax(output,dim=1)
  value,index=torch.max(probs,dim=1)
  if value<threshold:
    print("Dont know")
  print(list(vocab.keys())[index])

In [ ]:
predict(model,"Boiling point of water")

Dont know
h2o
